In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.utils import shuffle

In [ ]:
def pad_and_concat(sequences):  # sequences shape: [batch_size, len, dims...] -> ([batch_size, maxlen, dims...], [len])
    arrays = [np.asarray(seq) for seq in sequences]
    lengths = np.asarray([array.shape[0] for array in arrays], dtype=np.int32)
    maxlen = np.max(lengths)
    arrays = [np.pad(array, [(0, maxlen - array.shape[0]), (0, 0)], 'constant', constant_values=0) for array in arrays]
    return np.asarray(arrays), lengths
    

In [ ]:
SAMPLE = 200
VARIABLE_NUM = 4
EMBEDDING_SIZE = 8
CLAUSE_SIZE = 3
CLAUSE_NUM = 30
LSTM_STATE_SIZE = 8
batch_size = 6

In [ ]:
def assert_shape(matrix, shape: list):
    act_shape = matrix.get_shape().as_list()
    assert act_shape == shape, "got shape {}, expected {}".format(act_shape, shape)

In [ ]:
class Graph:
    def __init__(self):
        self.inputs = tf.placeholder(tf.int32, shape=(batch_size, None, CLAUSE_SIZE), name='inputs')
        self.lengths = tf.placeholder(tf.int32, shape=(batch_size,), name='lengths')
        self.policy_labels = tf.placeholder(tf.float32, shape=(batch_size, VARIABLE_NUM*2), name='labels')
        
        vars_ = tf.abs(self.inputs)
        signs = tf.cast(tf.sign(self.inputs), tf.float32)  # shape: [batch_size, None, CLAUSE_SIZE]

        embeddings = tf.Variable(tf.random_uniform([VARIABLE_NUM + 1, EMBEDDING_SIZE], -1., 1), name='embeddings')

        var_embeddings = tf.nn.embedding_lookup(embeddings, vars_)
        # var_embeddings shape: [None, None, CLAUSE_SIZE, EMBEDDING_SIZE]
        
        clause_preembeddings = tf.concat(
            [tf.reshape(var_embeddings, [batch_size, -1, CLAUSE_SIZE * EMBEDDING_SIZE]), 
             signs],
            axis=2)
        
        PREEMBEDDING_SIZE = EMBEDDING_SIZE * CLAUSE_SIZE + CLAUSE_SIZE
        assert_shape(clause_preembeddings, 
                     [batch_size, None, PREEMBEDDING_SIZE])
        
        clause_w = tf.Variable(tf.random_normal(
            [PREEMBEDDING_SIZE, EMBEDDING_SIZE]), name='clause_w')
        clause_b = tf.Variable(tf.random_normal([EMBEDDING_SIZE]), name='clause_b')
        clause_embeddings = tf.reshape(tf.sigmoid(
            tf.reshape(clause_preembeddings, [-1, PREEMBEDDING_SIZE]) @ clause_w + clause_b), 
                                       [batch_size, -1, EMBEDDING_SIZE])
        # shape: [None, None, EMBEDDING_SIZE]
        
        lstm = tf.contrib.rnn.BasicLSTMCell(LSTM_STATE_SIZE)
        hidden_state = tf.zeros([batch_size, LSTM_STATE_SIZE])
        current_state = tf.zeros([batch_size, LSTM_STATE_SIZE])
        state = hidden_state, current_state
        
        _, lstm_final_state = tf.nn.dynamic_rnn(lstm, clause_embeddings, dtype=tf.float32, 
                                               sequence_length=self.lengths
                                               )
        formula_embedding = lstm_final_state.h
            
        assert_shape(formula_embedding, [batch_size, LSTM_STATE_SIZE])
            
        softmax_w = tf.Variable(tf.random_normal([LSTM_STATE_SIZE, VARIABLE_NUM*2]), name='softmax_w')
        softmax_b = tf.Variable(tf.random_normal([VARIABLE_NUM*2]), name='softmax_b')
        
        self.policy_logits = formula_embedding @ softmax_w + softmax_b
        self.loss = tf.losses.sigmoid_cross_entropy(self.policy_labels, self.policy_logits) 
        self.policy_probabilities = tf.sigmoid(self.policy_logits)
        self.policy_error = tf.reduce_mean(tf.abs(
            tf.round(self.policy_probabilities) - self.policy_labels))
        
        tf.summary.scalar("loss", self.loss)
        tf.summary.scalar("policy_error", self.policy_error)

tf.reset_default_graph()
model = Graph()

In [ ]:
np.set_printoptions(precision=2, suppress=True)

In [ ]:
from cnf import get_random_kcnfs

In [ ]:
def sat_array():
    for k in range(1, 6):
        for var_num in range(2, 5):
            for clause_num in range(3, 10):
                sat = {True: 0, False: 0}
                for _ in range(100):
                    sat[cnf.get_random_kcnf(k, var_num, clause_num).satisfiable()] += 1
                print(k, var_num, clause_num, sat)
# sat_array()

In [ ]:
print("info:", SAMPLE, CLAUSE_SIZE, VARIABLE_NUM, CLAUSE_NUM)

cnfs = get_random_kcnfs(SAMPLE, CLAUSE_SIZE, VARIABLE_NUM, CLAUSE_NUM, min_clause_number=3)
plt.title("original cnfs: {}".format(len(cnfs)))
plt.hist([len(cnf.clauses) for cnf in cnfs], bins=range(CLAUSE_NUM+2))
plt.xlabel("num of clauses")
plt.show()
cnfs = [cnf for cnf in cnfs if cnf.satisfiable()]
plt.title("sat cnfs: {}".format(len(cnfs)))
plt.hist([len(cnf.clauses) for cnf in cnfs], bins=range(CLAUSE_NUM+2))
plt.xlabel("num of clauses")
plt.show()
print("#samples:", len(cnfs))
assert len(cnfs) * 2 > SAMPLE

In [ ]:
#labels = [cnf.satisfiable() for cnf in cnfs]
labels = []
for cnf in cnfs:
    correct_steps = cnf.get_correct_steps()
    label = []
    # for every variable, for true, then for false
    for v in range(1, VARIABLE_NUM+1):
        for sv in [v, -v]:
            result = 1.0 if sv in correct_steps else 0.0
            label.append(result)
    labels.append(label)
assert all(len(label) == 2*VARIABLE_NUM for label in labels)
sat_steps = sum(sum(label) for label in labels)
unsat_steps = len(cnfs)*2*VARIABLE_NUM - sat_steps
print("SAT/UNSAT steps: {} / {}".format(sat_steps, unsat_steps))
assert unsat_steps * 4 > sat_steps
print("labels generated")

In [ ]:
plt.title("num of clauses vs %correct steps")
plt.scatter([len(cnf.clauses) for cnf in cnfs], [sum(label)/2./VARIABLE_NUM for label in labels], alpha=0.5)
plt.xlabel("num of clauses")
plt.ylabel("percent of correct steps")
plt.show()

In [ ]:
cnfs_train, cnfs_test, labels_train, labels_test = train_test_split(cnfs, labels)

In [ ]:
def chunks(lists, chunk_size):
    return [[it[i:i + chunk_size] for it in lists] for i in range(0, len(lists[0]), chunk_size)]

In [ ]:
import datetime
merged_summaries = tf.summary.merge_all()

SUMMARY_DIR = "summaries"
MODEL_NAME = "onlypolicy"
DATESTR = datetime.datetime.now().strftime("%y-%m-%d-%H%M%S")
SUMMARY_PREFIX = SUMMARY_DIR + "/" + MODEL_NAME + "-" + DATESTR
train_writer = tf.summary.FileWriter(SUMMARY_PREFIX + "-train")
test_writer = tf.summary.FileWriter(SUMMARY_PREFIX + "-test")

NAME = "{} {}-SATs with {} vars and {} clauses".format(len(cnfs), CLAUSE_SIZE, VARIABLE_NUM, CLAUSE_NUM)
glob_losses = []
glob_accs = []
glob_test_losses = []
glob_test_accs = []

with tf.Session() as sess:
    train_op = tf.train.AdamOptimizer(learning_rate=0.01).minimize(model.loss)
    sess.run(tf.global_variables_initializer())
    
    EPOCHS = 10000
    PRINTS = 10
    global_step = 0
    for epoch_num in range(EPOCHS):
        print("Epoch", epoch_num)
        losses = []
        accs = []
        test_losses = []
        test_accs = []
        cnfs_train, labels_train = shuffle(cnfs_train, labels_train)
        for (batch_cnfs, batch_labels) in chunks((cnfs_train, labels_train), batch_size):
            if len(batch_cnfs) < batch_size:
                continue
            inputs, lengths = pad_and_concat([cnf.clauses for cnf in batch_cnfs])
            summary, _, loss, probs = sess.run([merged_summaries, train_op, model.loss, model.policy_probabilities], feed_dict={
                model.inputs: inputs,
                model.policy_labels: batch_labels,
                model.lengths: lengths
            })
            train_writer.add_summary(summary, global_step)
            
            global_step += 1

        
        for (batch_cnfs, batch_labels) in chunks((cnfs_test, labels_test), batch_size):
            if len(batch_cnfs) < batch_size:
                continue
            inputs, lengths = pad_and_concat([cnf.clauses for cnf in batch_cnfs])
            summary, loss, probs = sess.run([merged_summaries, model.loss, model.policy_probabilities], feed_dict={
                model.inputs: inputs,
                model.policy_labels: batch_labels,
                model.lengths: lengths
            })
            test_writer.add_summary(summary, global_step)
            
